# Read raw data

In [0]:
df_recommendation = spark.read.table("anac_data.raw_csv_data.recomendacao")
df_significant_factor = spark.read.table("anac_data.raw_csv_data.fator_contribuinte")
df_type_occurrence = spark.read.table("anac_data.raw_csv_data.ocorrencia_tipo")
df_occurrence = spark.read.table("anac_data.raw_csv_data.ocorrencia")
df_aircraft = spark.read.table("anac_data.raw_csv_data.aeronave")

# Schema Creation

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS anac_aeronautical_occurrences_in_brazilian_civil_aviation;
CREATE SCHEMA IF NOT EXISTS anac_aeronautical_occurrences_in_brazilian_civil_aviation.bronze_layer;

# Save delta tables in bronze layer


In [0]:
dataframes = {
    "gov_recommendation": df_recommendation,
    "gov_significant_factor": df_significant_factor,
    "gov_type_occurrence": df_type_occurrence,
    "gov_occurrence": df_occurrence,
    "gov_aircraft": df_aircraft
}

for table_name, dataframe in dataframes.items():
    dataframe.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable(f"anac_aeronautical_occurrences_in_brazilian_civil_aviation.bronze_layer.{table_name}")

print("All tables saved in bronze layer.")